# Point Cloud Demonstrations

## Registration

Registers two point clouds with slight offset.


In [1]:
import open3d as o3d
import numpy as np
import copy
import random
import copy
import scipy.spatial
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import scipy as sp
import utils
import correspondance as corr
import scenery as sce

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

# load point clouds
base = o3d.io.read_point_cloud("./Point Clouds/Test 1/1.ply")
source = o3d.io.read_point_cloud("./Point Clouds/Test 1/8.ply")

# get 4x4 transformation matrix
trans = corr.register_clouds(base, source)

# show original clouds
o3d.visualization.draw_geometries([base, source])

# show aligned clouds
draw_registration_result_original_color(source, base, trans)


## Normals of point cloud
Shows the histogram of normals for the cloud. See utils module - running in notebook is much slower

## Outlier removal
Removes outliers and scatter artifacts from the pointcloud

In [ ]:
cl, ind=utils.clean_cloud("./Point Clouds/Test 1/1.ply", view=True)

## Cleaned and combined 8 clouds

In [ ]:
base = o3d.io.read_point_cloud("./Point Clouds/Test 1/test_1.ply")
o3d.visualization.draw_geometries([base])

# Region Growing Segmentation
Demonstrates region growing algorithm for downsampled cloud

In [ ]:
cl = o3d.io.read_point_cloud("./Point Clouds/Test 1/test_1.ply")
pcd = cl.voxel_down_sample(0.01)
pcd.estimate_normals()

planes_list = corr.region_grow(pcd)

for plane in planes_list:
    colour = [random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]
    for i in plane:
        pcd.colors[i] = colour

o3d.visualization.draw_geometries([pcd])

# extract walls which is the largest object in the cloud

walls = planes_list[np.argmax(np.array([len(i) for i in planes_list]))]
walls = pcd.select_down_sample(walls)

## Isolating each wall plane

In [ ]:
planes_list, plane_normals = corr.region_grow(walls, 0.8, True)

colours = [[1,0,0],[0,1,0],[0,0,1]]

for plane, colour in zip(planes_list, colours):
    for i in plane:
        walls.colors[i] = colour

o3d.visualization.draw_geometries([walls])

print(plane_normals)


## SVD of Walls
Principle vectors pick out the normals of the wall

In [ ]:
plane = planes_list[1]
wall_cp = copy.deepcopy(walls)
plane = wall_cp.select_down_sample(plane)
points = np.asarray(plane.points)
points -= np.mean(points, axis=0)
u, s, v = np.linalg.svd(points)

normal = np.matmul(np.linalg.pinv(points), np.ones(points.shape[0]))
normal /= np.linalg.norm(normal)

o3d.visualization.draw_geometries([plane, utils.create_vector_graph(v), utils.create_vector_graph([normal])])

## Match to plane in full size cloud  (deprec)

In [ ]:
# now match to the original full size point cloud

cl.paint_uniform_color([0.5,0.5,0.5])
print(plane_normals)
for norm, colour in zip(plane_normals, colours):
    dot_match = np.abs(1-np.asarray(cl.points).dot(norm))
    for i in range(len(dot_match)):
        if dot_match[i] <0.04:
            cl.colors[i] = colour

# add normals for clearer viewing
if not cl.has_normals():
    cl.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
o3d.visualization.draw_geometries([cl])

## DBSCAN Methods (deprec)

In [ ]:
pcd = o3d.io.read_point_cloud("./Point Clouds/Test 1/test_1.ply")
pcd = pcd.voxel_down_sample(0.005)
pcd.estimate_normals()

isolated= corr.remove_planes(pcd)

labels = np.array(isolated.cluster_dbscan(eps=0.01, min_points=40, print_progress=True))
colours = np.random.rand(labels.max()+1, 3)
for i in range(len(labels)):
    if labels[i] == -1:
        isolated.colors[i] = [0,0,0]
        continue
    isolated.colors[i] = colours[labels[i]]
outliers = np.where(labels == -1)[0]
isolated = isolated.select_down_sample(outliers, invert=True)

o3d.visualization.draw_geometries([isolated])


## Merging model clouds


In [ ]:
scale = 1

cl1 = o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Commander/1.3.ply")
cl1 = corr.isolate_model(cl1)
plane1 = utils.create_origin_plane(100)
cl1 += plane1
cl1.scale(scale)

for i in range(4):
    print(i)
    cl2 = o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Commander/1.{}.ply".format(i+2))
    cl2 = corr.isolate_model(cl2)
    temp_cl = cl2 + plane1
    
    temp_cl.scale(scale)
    cl2.scale(scale)
    print("Registering")
    tran = corr.register_clouds(cl1, temp_cl, voxel_radius=[0.01, 0.005, 0.0001], max_iter=[1000,1000,10000])
    
    cl2 = cl2.transform(tran)
    cl1 += cl2
    cl1.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
    
o3d.visualization.draw_geometries([cl1])

# Reference models

In [ ]:
cmdr = o3d.io.read_point_cloud("./Point Clouds/Commander Ref.ply")
brdsd = o3d.io.read_point_cloud("./Point Clouds/Broadside Ref.ply")
o3d.visualization.draw_geometries([cmdr])
o3d.visualization.draw_geometries([brdsd])

# Recognising Models

In [ ]:
isolated = o3d.io.read_point_cloud("isolated.ply")
o3d.visualization.draw_geometries([isolated])

labels = np.array(isolated.cluster_dbscan(eps=0.01, min_points=40, print_progress=True))
targets = []
print(Counter(labels))
for i in range(labels.max()+1):
    obj = np.where(labels==i)[0]
    obj = isolated.select_down_sample(obj)
    obj = obj.translate(-obj.get_center())
    targets.append(obj)
    print(len(obj.points))
    print(obj.get_oriented_bounding_box().volume())

In [ ]:
# for i in targets:
#     print(i.get_oriented_bounding_box().volume())
cmdr = o3d.io.read_point_cloud("./Point Clouds/Commander Ref.ply")
brdsd = o3d.io.read_point_cloud("./Point Clouds/Broadside Ref.ply")

cmdr_vol = cmdr.get_oriented_bounding_box().volume()
brdsd_vol = brdsd.get_oriented_bounding_box().volume()
print("Ref volumes:")
print("Commander: {}".format(cmdr.get_oriented_bounding_box().volume()))
print("Broadside: {}".format(brdsd.get_oriented_bounding_box().volume()))
print(np.asarray(cmdr.get_oriented_bounding_box().get_box_points()))

# print("Commander matching")
# for pcd in targets:
#     vol = pcd.get_oriented_bounding_box().volume()
#     if vol > 0.3*cmdr_vol and vol < cmdr_vol:
#         o3d.visualization.draw_geometries([pcd])
        
# print("Broadside matching")
# for pcd in targets:
#     vol = pcd.get_oriented_bounding_box().volume()
#     if vol > 0.3*brdsd_vol and vol < brdsd_vol:
#         o3d.visualization.draw_geometries([pcd])

## Getting building axis

In [ ]:
# use this function in the table scenes, it will produce for the buildings the normal histograms

def matching(pcd, labels):
    # define fp to reference models
    refs = {}
    refs["Commander"] = o3d.io.read_point_cloud("./Point Clouds/Commander Ref.ply")
    refs["Broadside"] = o3d.io.read_point_cloud("./Point Clouds/Broadside Ref.ply")
    ref_vols = np.array([i.get_oriented_bounding_box().volume() for j, i in refs.items()])

    # go through all clusters that have been labelled but not classified as
    # outliers or the table plane
    for i in range(labels.max()):  # note that this will go up to but not include table
        cluster = np.where(labels == i)[0]
        cluster = pcd.select_down_sample(cluster)
        vol = cluster.get_oriented_bounding_box().volume()

        # first filter possible matches based on size
        matches = []
        if vol > np.max(ref_vols): # potentially item of scenery, cluster more finely
            print("Scenery cluster")
            cluster.estimate_normals()
            cluster.normalize_normals()
            planes, normals = corr.region_grow(cluster, find_planes=True)
            lens = np.array([len(i) for i in planes])
            normals = np.multiply(lens, normals.T).T
            vecs = utils.hist_normals(normals, bin_size=0.95)
            vecs = utils.create_vector_graph(vecs)
            vecs2 = utils.hist_normals(np.asarray(cluster.normals), bin_size=0.95)
            vecs2 = utils.create_vector_graph(vecs2)
            vecs2.paint_uniform_color([1,0,0])
            o3d.visualization.draw_geometries([cluster, vecs, vecs2, cluster.get_axis_aligned_bounding_box()])

                
clouds = []
for i in range(2,6):
    clouds.append(o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Board 2/7.{}.ply".format(i)))

print("Segmenting")
pcd = clouds[0]
labels, norm = corr.segment(pcd)
inliers = np.where(labels != -1)[0]
pcd = pcd.select_down_sample(inliers)
labels = np.array([i for i in labels if i !=-1])
T = utils.align_vectors(norm, np.array([0, 1, 0]))
pcd = pcd.transform(T)
pcd.translate(-pcd.get_center())
dist = np.mean(np.asarray(pcd.points)[np.where(labels == labels.max())[0]], axis=0)
pcd.translate(np.array([0,-1,0])*dist[1])

matching(pcd,labels)

In [ ]:
%matplotlib inline

def lines(x, n):
    return 1/n[1] - n[0]*x/n[1]

# This will plot the best fit lines for the corners
def matching2(pcd, labels):
    # define fp to reference models
    refs = {}
    refs["Commander"] = o3d.io.read_point_cloud("./Point Clouds/Commander Ref.ply")
    refs["Broadside"] = o3d.io.read_point_cloud("./Point Clouds/Broadside Ref.ply")
    ref_vols = np.array([i.get_oriented_bounding_box().volume() for j, i in refs.items()])

    # go through all clusters that have been labelled but not classified as
    # outliers or the table plane
    for i in range(labels.max()):  # note that this will go up to but not include table
        cluster = np.where(labels == i)[0]
        cluster = pcd.select_down_sample(cluster)
        vol = cluster.get_oriented_bounding_box().volume()

        # first filter possible matches based on size
        matches = []
        if vol > np.max(ref_vols): # potentially item of scenery, cluster more finely
            print("Scenery cluster")
            points = np.asarray(cluster.points)
            points = np.delete(points, 1, axis=1)
            points = np.concatenate((points, np.ones((points.shape[0], 1))), axis=1)
            hull = scipy.spatial.ConvexHull(points[:,:2])
            hull_pts = points[hull.vertices]
            while len(hull_pts) < 50:
                points = np.delete(points, hull.vertices, axis=0)
                hull = scipy.spatial.ConvexHull(points[:,:2])
                hull_pts =np.append(hull_pts, points[hull.vertices], axis=0)
                x = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(hull_pts)).voxel_down_sample(0.01)
                hull_pts = np.asarray(x.points)
            hull_pts = hull_pts[:,:2]

            R = np.array([[0, -1],
                          [1, 0]])
            n, alpha, ind1, ind2, cost = utils.fit_corner2(hull_pts)
            line_x = np.linspace(hull_pts[:, 0].min(), hull_pts[:, 0].max())
            y1 = lines(line_x, n)
            y2 = lines(line_x, R.dot(n)/alpha)
#             plt.scatter(points[:,0], points[:,1])
            for i in [ind1, ind2]:
                plt.scatter(hull_pts[i, 0], hull_pts[i,1])
            corner = np.sum(np.linalg.inv(np.array([n, R.dot(n)/alpha])), axis=1)
            print(corner)
            plt.scatter(corner[0], corner[1])
            plt.plot(line_x, y1)
            plt.plot(line_x, y2)
            xrange = hull_pts[:,0].max()-hull_pts[:,0].min()
            plt.xlim(hull_pts[:,0].min()-0.1*xrange, hull_pts[:,0].max()+0.1*xrange,)
            yrange = hull_pts[:,1].max()-hull_pts[:,1].min()
            plt.ylim(hull_pts[:,1].min()-0.1*yrange, hull_pts[:,1].max()+0.1*yrange,)
            plt.tight_layout()
            plt.show()


clouds = []
for i in range(2,6):
    clouds.append(o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Board 2/7.{}.ply".format(i)))

# for i in range(1,8):
#     clouds.append(o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Board 1/6.{}.ply".format(i)))

for pcd in clouds:
    labels, norm = corr.segment(pcd)
    R = utils.align_vectors(norm, np.array([0,1,0]))
    pcd.transform(R)
    pcd.translate(-pcd.get_center())
    table = np.where(labels == labels.max())
    points = np.asarray(pcd.points)
    table_pts = points[table]
    pcd.translate(np.array([0, -np.mean(table_pts, axis=1)[1], 0]))

    matching2(pcd, labels)


## Global alignment

In [7]:
clouds = []
for i in range(1,9):
    clouds.append(o3d.io.read_point_cloud("./Point Clouds/Layout 3/{}.ply".format(i)))

build_ref = o3d.io.read_triangle_mesh("./Point Clouds/Building Ref.ply")
build_ref = build_ref.compute_triangle_normals()

# for i in range(1,8):
#     clouds.append(o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Board 1/6.{}.ply".format(i)))
pcd = clouds[0]
labels, norm = corr.segment(pcd)
clouds[0] = corr.building_align(pcd, labels, norm)
base = clouds[0].select_down_sample(np.where(labels != -1)[0])
clouds[0] = base

for i in range(8):
    pcd = clouds[i]
    labels, norm = corr.segment(pcd)
    clouds[i] = corr.building_align(pcd, labels, norm)
    clouds[i] = pcd.select_down_sample(np.where(labels!=-1)[0])
    clouds[i].estimate_normals()

# o3d.visualization.draw_geometries(clouds)

for i in range(1,8):
    print(i)
    T = corr.register_clouds(base.voxel_down_sample(0.004), clouds[i].voxel_down_sample(0.004))
    theta = np.arctan2(-T[2,0], np.linalg.norm(T[2,1:3]))
    R = np.array([[np.cos(theta), 0, np.sin(theta)],
                  [0,             1, 0            ],
                  [-np.sin(theta), 0, np.cos(theta)]])
    T = np.array(T)
    print(T)
    T[:3,:3]=R
    print(T)
    clouds[i].transform(T)
    base += clouds[i]

# base.estimate_normals()
# base = base.voxel_down_sample(0.003)
# labels, norm = corr.segment(base)
# base = utils.colour_labels(base, labels)

# for i in range(labels.max()):
#     cluster = np.where(labels == i)[0]
#     cluster = base.select_down_sample(cluster)
#     points = np.asarray(cluster.points)
#     points = np.delete(points, 1, axis=1)
#     plt.scatter(points[:,0], points[:,1])

# building = 0
# max_vol = 0
# for i in range(labels.max()):  # note that this will go up to but not include table
#     cluster = np.where(labels == i)[0]
#     cluster = base.select_down_sample(cluster)
#     vol = cluster.get_oriented_bounding_box().volume()
#     # building will be largest cluster
#     if vol > max_vol:
#         max_vol = vol
#         building = cluster

# base = corr.building_align(base, labels, norm)
# print(np.cbrt(max_vol/ build_ref.get_oriented_bounding_box().volume()))
# build_ref.scale(np.cbrt(max_vol/ build_ref.get_oriented_bounding_box().volume()), center=False)
# # o3d.visualization.draw_geometries([building, build_ref])
o3d.visualization.draw_geometries([base])


1
[[ 9.88349062e-01 -5.34087979e-03 -1.52110509e-01 -7.00670004e-03]
 [ 5.34775835e-03  9.99985634e-01 -3.63887535e-04  9.94348843e-04]
 [ 1.52110267e-01 -4.53802338e-04  9.88363425e-01  2.85296524e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.88363530e-01  0.00000000e+00 -1.52110267e-01 -7.00670004e-03]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  9.94348843e-04]
 [ 1.52110267e-01  0.00000000e+00  9.88363530e-01  2.85296524e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
2
[[ 9.94597380e-01  7.05775233e-03 -1.03567562e-01 -8.18957723e-03]
 [-7.28541311e-03  9.99971805e-01 -1.82006271e-03 -6.29559041e-04]
 [ 1.03551796e-01  2.56476208e-03  9.94620756e-01  2.90222326e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.94624062e-01  0.00000000e+00 -1.03551796e-01 -8.18957723e-03]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -6.29559041e-04]
 [ 1.03551796e-01  0.00000000e+00  9.94624062e-01  2.90

True

## Full Scene Matching

In [ ]:
print("Loading References...")
models  = utils.open_refs()

In [ ]:
print("Extracting Target models...")
# base = base1.voxel_down_sample(0.004)
# labels, norm = corr.segment(base)

# get building for scaling
building = 0
max_vol = 0
for i in range(labels.max()):  # note that this will go up to but not include table
    cluster = np.where(labels == i)[0]
    cluster = base.select_down_sample(cluster)
    vol = cluster.get_oriented_bounding_box().volume()
    # building will be largest cluster
    if vol > max_vol:
        max_vol = vol
        building = cluster

points = np.asarray(building.points)
height = points[:, 1].max()-points[:, 1].min()
scale_factor = 20/height  # height of building
base.scale(scale_factor)

counts = Counter(labels)
counts = Counter(labels)
model_labels = [i for i, j in counts.items() if (i, j) not in counts.most_common(2)]
model_labels = [i for i in model_labels if i != -1]

targets = [base.select_down_sample(np.where(labels == i)[0]) for i in model_labels]
# targets = [base1.crop(i.get_axis_aligned_bounding_box()) for i in targets]

table = base.select_down_sample(np.where(labels == labels.max())[0])
table_color = np.mean(np.asarray(table.colors), axis=0)

result = corr.match_model(targets, models)
figures = []
for i, j in result:
    figures.append(models[i].transform(np.linalg.inv(j)))

figures.append(base)

o3d.visualization.draw_geometries(figures)

## Scene Object

In [ ]:
print("Loading Target Point Clouds...")
base1 = copy.deepcopy(base)
scene = sce.Scene(base1)
scene.show_scene()
o3d.visualization.draw_geometries([scene.cloud])
scene.show_top_view()


In [ ]:
direc = scene.minis[0].get_center() - scene.minis[1].get_center()
direc[1] = 0
print(np.linalg.norm(direc)/2.54)

In [ ]:
com = copy.deepcopy(models["Fireblade"])
triangles = np.asarray(com.triangles)
vertices = np.asarray(com.vertices)

print(triangles)
print(vertices)
vertices[:,2] = 0
com.vertices = o3d.utility.Vector3dVector(vertices)
com.merge_close_vertices(0.005)
o3d.visualization.draw_geometries([com])

 # Results
 ## Corner Matching 
 Creates subplots for a number of sampled buildings, showing the intersection lines. Parameters need to be fine tuned on the new fit_corners since the scaling ahs been removed no outliers seem to appear.

In [ ]:
data = []
ax_range = 0
x_mins = []
y_mins = []
fig, axes = plt.subplots(2,2, figsize=(8,8))
axes = axes.flatten()
for j in range(2,6):
    print(j)
    pcd = o3d.io.read_point_cloud("./Point Clouds/Layout 2/{}.ply".format(j))
    pcd = pcd.voxel_down_sample(0.004)
    labels, norm = corr.segment(pcd)

    R = utils.align_vectors(norm, np.array([0, 1, 0]))
    pcd.transform(R)
    pcd.translate(-pcd.get_center())
    table = np.where(labels == labels.max())[0]
    points = np.asarray(pcd.points)
    table_pts = points[table]
    pcd.translate(np.array([0, -np.mean(table_pts, axis=0)[1], 0]))

    # find the building cluster
    building = 0
    max_vol = 0
    for i in range(labels.max()):  # note that this will go up to but not include table
        cluster = np.where(labels == i)[0]
        cluster = pcd.select_down_sample(cluster)
        vol = cluster.get_oriented_bounding_box().volume()
        # building will be largest cluster
        if vol > max_vol:
            max_vol = vol
            building = cluster

    # fit building corners
    # Project points onto x-z plane
    points = np.asarray(building.points)

    # use height to scale length when selecting hull points
    height = points[:,1].max()

    # get rid of any base that might be included
    points = points[np.where(points[:,1] > 0.2*points[:,1].max())[0]]
    points = points[:, ::2]
    points = np.concatenate((points, np.ones((points.shape[0], 1))), axis=1)

    # downsample top down view as compressed slice will be dense and hull sparse
    x = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points)).voxel_down_sample(height/25)
    points = np.asarray(x.points)
    hull = sp.spatial.ConvexHull(points[:, :2])
    hull_pts = points[hull.vertices]

    # iterate convex hulls until we have at least 50 points to fit
    while len(hull_pts) < 50:
        points = np.delete(points, hull.vertices, axis=0)
        hull = sp.spatial.ConvexHull(points[:, :2])
        hull_pts = np.append(hull_pts, points[hull.vertices], axis=0)
    hull_pts = hull_pts[:, :2]

    # 90 degree rotation to get second normal from original normal
    R = np.array([[0, -1],
                  [1, 0]])

    R45 = np.array([[1., -1.],  # 90 degree rotation matrix
                    [1., 1.]])
    R45 *= 1/np.sqrt(2)

    n, alpha, inliers1, inliers2, cost = utils.fit_corner(hull_pts, height)
    print("Alpha: {}".format(alpha))
    n2 = (R @ n)/alpha
    x = np.linspace(-50, 50, 5)
    y1 = -(x*n[0] -1)/n[1]
    y2 = -(x*n2[0] -1)/n2[1]

    c = np.linalg.inv(np.array([n, n2])) @ np.ones(2)
    bisec = R45 @ n
    bisec /= np.linalg.norm(bisec)
    y3 = -(x*bisec[0]- c @ bisec)/bisec[1]

    points = points[:,:2]
    axes[j-2].plot(x, y1)
    axes[j-2].plot(x, y2)
    axes[j-2].plot(x, y3, ls='--', color="C3")
    l1 = inliers1
    l2 = inliers2
    axes[j-2].scatter(points[:,0], points[:,1], color="C2", label="Non-hull points")
    axes[j-2].scatter(hull_pts[:,0], hull_pts[:,1], color="C3", label="Outliers")
    axes[j-2].scatter(l1[:,0], l1[:,1], label="Convex hull 1")
    axes[j-2].scatter(l2[:,0], l2[:,1], label="Convex hull 2")
    axes[j-2].set_title(r"$n = [{:.3f},{:.3f}]$, $\alpha={:.2f}$".format(*n, alpha))

    # n, d, inliers1, inliers2, cost = utils.fit_corner2(hull_pts, height)
    # n2 = (R @ n)
    # x = np.linspace(-50, 50, 5)
    # y1 = -(x*n[0] -d[0])/n[1]
    # y2 = -(x*n2[0] -d[1])/n2[1]
    #
    # c = np.linalg.inv(np.array([n, n2])) @ d
    # bisec = R45 @ n
    # bisec /= np.linalg.norm(bisec)
    # y3 = -(x*bisec[0]- c @ bisec)/bisec[1]
    #
    # points = points[:,:2]
    # axes[j-2].plot(x, y1)
    # axes[j-2].plot(x, y2)
    # axes[j-2].plot(x, y3, ls='--', color="C3")
    # l1 = inliers1
    # l2 = inliers2
    # axes[j-2].scatter(points[:,0], points[:,1], color="C2", label="Non-hull points")
    # axes[j-2].scatter(hull_pts[:,0], hull_pts[:,1], color="C3", label="Outliers")
    # axes[j-2].scatter(l1[:,0], l1[:,1], label="Convex hull 1")
    # axes[j-2].scatter(l2[:,0], l2[:,1], label="Convex hull 2")

    x_mins.append(hull_pts[:,0].min())
    y_mins.append(hull_pts[:,1].min())

    if hull_pts[:,0].max() - x_mins[-1] > ax_range:
        ax_range = hull_pts[:,0].max() - x_mins[-1]

    if hull_pts[:,1].max() - y_mins[-1] > ax_range:
        ax_range = hull_pts[:,1].max() - y_mins[-1]

for i in range(len(axes)):
    axes[i].set_xlim(x_mins[i]-0.1*ax_range, x_mins[i]+1.1*ax_range)
    axes[i].set_ylim(y_mins[i]-0.1*ax_range, y_mins[i]+1.1*ax_range)

plt.legend()
plt.tight_layout()
plt.show()

## Normal histograms
Creates plots to put the normals into a spherical histogram

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(4,8), sharex=True, sharey=True)
axes = axes.flatten()

# fig3d = plt.figure(figsize=(5.5,5))
# ax3d = fig3d.add_subplot(111, projection='3d')

# base = o3d.io.read_point_cloud("./Point Clouds/Building_1000.ply")
base = o3d.io.read_point_cloud("./building.ply")
vecs = utils.hist_normals(np.asarray(base.normals), 0.95)
mags = np.linalg.norm(vecs, axis=1)

for i in range(len(vecs)):
    vecs[i,:] /= mags[i]

norms = np.asarray(base.normals)
for i in range(len(norms)):
    norms[i] *= np.sign(norms[i,np.argmax(abs(norms[i]))])

angle = 90
angle *= np.pi/180
R45 = np.array([[np.cos(angle), 0, np.sin(angle)],
                [0,1,0],
                [-np.sin(angle), 0,np.cos(angle)]])

# norms = norms @ R45
x = np.sqrt(2/(1-norms[:,0]))*norms[:,1]
y = np.sqrt(2/(1-norms[:,0]))*norms[:,2]

# phi = np.arcsin(norms[:,2])
# theta = np.arcsin(norms[:,0]/np.cos(phi))
# x = theta*np.sin(phi)
# y = theta*np.cos(phi)
im = axes[0].hist2d(norms[:,0], norms[:,1], bins=50, cmap=mpl.cm.jet, norm=mpl.colors.LogNorm())
axes[1].hist2d(norms[:,2], norms[:,1], bins=50, cmap=mpl.cm.jet, norm=mpl.colors.LogNorm())
axes[0].set_title("xy plane")
axes[1].set_title("zy plane")

# n = 1000000
# data = np.empty((n, 3))
# data[:,0] = np.random.normal(0, 1, n)
# data[:,1] = np.random.normal(0, 1.3, n)
# data[:,2] = np.random.normal(1, 1.2, n)
# norms[:,0]=1
# norms[:,1]=theta
# norms[:,2]=phi
# h = physt.special.spherical_histogram(norms)
# globe = h.projection("theta", "phi")
# # globe.plot()
# globe.plot.globe_map(density=True, figsize=(7, 7), cmap="jet")


# ax3d.scatter(vecs[:,0], vecs[:,1], vecs[:,2], marker='o', s=2000*mags**1.2, depthshade=True, label="Full Building")

for j in range(2,5):
    print(j)
    pcd = o3d.io.read_point_cloud("./Point Clouds/Layout 2/{}.ply".format(j))
    # if not pcd.has_normals():
    pcd.estimate_normals()
    # pcd = pcd.voxel_down_sample(0.004)
    labels, norm = corr.segment(pcd)

    R = utils.align_vectors(norm, np.array([0, 1, 0]))
    pcd.transform(R)
    pcd.translate(-pcd.get_center())
    table = np.where(labels == labels.max())[0]
    points = np.asarray(pcd.points)
    table_pts = points[table]
    pcd.translate(np.array([0, -np.mean(table_pts, axis=0)[1], 0]))

    # find the building cluster
    building = 0
    max_vol = 0
    for i in range(labels.max()):  # note that this will go up to but not include table
        cluster = np.where(labels == i)[0]
        cluster = pcd.select_down_sample(cluster)
        vol = cluster.get_oriented_bounding_box().volume()
        # building will be largest cluster
        if vol > max_vol:
            max_vol = vol
            building = cluster

    normals = np.asarray(building.normals)
    normals2 = normals[np.where(np.linalg.norm(normals, axis=1) > 0.9)[0]]
    vecs = utils.hist_normals(normals2, 0.95)
    mags = np.linalg.norm(vecs, axis=1)
    for i in range(len(vecs)):
        vecs[i,:] /= mags[i]

    for i in range(len(normals)):
        normals[i] *= np.sign(normals[i,np.argmax(abs(normals[i]))])
    # normals = normals @ R45
    x = np.sqrt(2/(1-normals[:,0]))*normals[:,1]
    y = np.sqrt(2/(1-normals[:,0]))*normals[:,2]
    # phi = np.arcsin(normals[:,2])
    # theta = np.arcsin(normals[:,0]/np.cos(phi))
    # x = theta*np.sin(phi)
    # y = theta*np.cos(phi)

    # ax3d.scatter(vecs[:,0], vecs[:,1], vecs[:,2], marker='o', s=2000*mags**1.2, depthshade=True, label="View {}".format(j-1))
    axes[2*(j-1)].hist2d(normals[:,0], normals[:,1], bins=50, cmap=mpl.cm.jet, norm=mpl.colors.LogNorm())
    axes[2*(j-1)+1].hist2d(normals[:,2], normals[:,1], bins=50, cmap=mpl.cm.jet, norm=mpl.colors.LogNorm())
    axes[(j-1)].set_xlim(-1,1)
    axes[(j-1)].set_ylim(-1,1)

# ax3d.set_xlabel('X')
# ax3d.set_ylabel('Y')
# ax3d.set_zlabel('Z')
# ax3d.set_xlim(-1,1)
# ax3d.set_ylim(-1,1)
# ax3d.set_zlim(-1,1)
# ax3d.view_init(elev=29, azim=45)
# lgnd = plt.legend()
# for i in range(len(lgnd.legendHandles)):
#     lgnd.legendHandles[i]._sizes = [30]
plt.tight_layout()
plt.show()